# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 2

Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE)

Septembre 2021


# 1. Assemblies

More contiguous genome assemblies can be generated using long sequencing read but assembly is not a quiet pace. Eukaryotic genomes assembly is a complex task (large genome sizes, high rates of repeated sequences, high heterozygosity levels and even polyploidy). While prokaryotic genomes may appear less challenging, specific features such as circular DNA molecules, must be taken into consideration to achieve high quality assembly.

* For assembly, ONT recommend sequencing a human genome to a minimum depth of 30x of 25–35 kb reads.
However, sequencing to a depth of 60x is advisable to obtain the best assembly metrics. We also recommend basecalling in high accuracy mode. Greatest contig N50 is usually obtained with Shasta and Flye. Polishing/Correction is also recommended (Racon and Medaka). https://nanoporetech.com/sites/default/files/s3/literature/human-genome-assembly-workflow.pdf


* Long reads simplify genome assembly, with the ability to span repeat-rich sequences (characteristic of  antimicrobial resistance genes) and structural variants. Nanopore sequencing also shows a lack of
bias in GC-rich regions, in contrast to other sequencing platforms. To perform microbial genome assembly, we suggest using the third-party de novo assembly tool Flye. We also recommend one round of polishing with Medaka. https://nanoporetech.com/sites/default/files/s3/literature/microbial-genome-assembly-workflow.pdf 

Flye  https://github.com/fenderglass/Flye 

Canu  https://canu.readthedocs.io/en/latest/quick-start.html

Miniasm  https://github.com/lh3/miniasm + Minipolish version https://github.com/rrwick/Minipolish

Shasta  https://github.com/chanzuckerberg/shasta

Smartdenovo  https://github.com/ruanjue/smartdenovo

Raven  https://github.com/lbcb-sci/raven

### 1.1 Assembly using Flye

We are going to assembly some Clones by using Flye https://github.com/fenderglass/Flye

Flye generates the concatenation of multiple disjoint genomic segments called disjointigs to build a repeat graph. Reads are mapped to this repeat graph to resolve conflicts (unbridged repeats) and output contigs.

In [23]:
# declare your clone number
CLONE="Clone10"
# create a repertory for save flye assembly results
cd ~/SG-ONT-2021/RESULTS/
pwd

(medaka) (medaka) (medaka) (medaka) /home/jovyan/SG-ONT-2021/RESULTS
(medaka) 

: 1

In [3]:
# Run flye
conda activate flye
time flye --nano-raw ../DATA/${CLONE}/ONT/${CLONE}.fastq.gz --genome-size 1000000 --out-dir FLYE --threads 4
conda deactivate

(base) (flye) [2021-09-24 14:10:39] INFO: Starting Flye 2.9-b1768
[2021-09-24 14:10:39] INFO: >>>STAGE: configure
[2021-09-24 14:10:39] INFO: Configuring run
[2021-09-24 14:10:44] INFO: Total read length: 166436048
[2021-09-24 14:10:44] INFO: Input genome size: 1000000
[2021-09-24 14:10:44] INFO: Estimated coverage: 166
[2021-09-24 14:10:44] INFO: Reads N50/N90: 22325 / 7615
[2021-09-24 14:10:44] INFO: Minimum overlap set to 8000
[2021-09-24 14:10:44] INFO: >>>STAGE: assembly
[2021-09-24 14:10:44] INFO: Assembling disjointigs
[2021-09-24 14:10:44] INFO: Reading sequences
[2021-09-24 14:10:52] INFO: Counting k-mers:
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-24 14:12:03] INFO: Filling index table (1/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-24 14:12:25] INFO: Filling index table (2/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-24 14:13:31] INFO: Extending reads
[2021-09-24 14:16:19] INFO: Overlap-based coverage: 122
[2021-09-24 14:16:19] INFO: Median

: 1

### 1.2 Assembly using Raven

In [5]:
# create a repertory for save raven assembly results
mkdir ~/SG-ONT-2021/RESULTS/RAVEN
cd ~/SG-ONT-2021/RESULTS/RAVEN
pwd

(base) (base) (base) /home/jovyan/SG-ONT-2021/RESULTS/RAVEN
(base) 

: 1

In [6]:
# Run raven
conda activate raven
time raven -p 0 --graphical-fragment-assembly ${CLONE}_raven.gfa -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz > ${CLONE}_raven.fasta
conda deactivate

(base) (raven) [raven::] loaded 11235 sequences 4.058354s
[raven::Graph::Construct] minimized 0 - 11235 / 11235 2.603075s
[raven::Graph::Construct] mapped sequences 10.619918s
[raven::Graph::Construct] annotated piles 0.100557s
[raven::Graph::Construct] removed contained sequences 0.033064s
[raven::Graph::Construct] removed chimeric sequences 0.006844s
[raven::Graph::Construct] reached checkpoint 0.083452s
[raven::Graph::Construct] minimized 0 - 214 / 214 0.860748s
[raven::Graph::Construct] mapped valid sequences 0.670266s
[raven::Graph::Construct] updated overlaps 0.000650s
[raven::Graph::Construct] removed false overlaps 0.009036s
[raven::Graph::Construct] stored 428 nodes 0.061790s
[raven::Graph::Construct] stored 3100 edges 0.001142s
[raven::Graph::Construct] reached checkpoint 0.182901s
[raven::Graph::Construct] 15.233442s
[raven::Graph::Assemble] removed transitive edges 0.002987s
[raven::Graph::Assemble] reached checkpoint 0.107995s
[raven::Graph::Assemble] removed tips and bubb

: 1

* How many contigs were obtained by Flye and Raven ? Please fill in results in the shared [file](https://lite.framacalc.org/9pd3-ont_sg_2021)
* Calculate first statistics about assemblies: N50 and lenght contig mean.

In [7]:
grep -c ^'>' ~/SG-ONT-2021/RESULTS/*/*.fasta

/home/jovyan/SG-ONT-2021/RESULTS/FLYE/assembly.fasta:4
/home/jovyan/SG-ONT-2021/RESULTS/RAVEN/Clone10_raven.fasta:3
(base) 

: 1

# 2. Polishing assemblies with Racon

Racon corrects raw contigs generated by rapid assembly methods with original ONT reads.

From 2 to 4 racon rounds are usually used by the community. 

Polish contigs assembled by Flye using two rounds of racon !.

In [14]:
conda activate racon
racon

(racon) [racon::] error: missing input file(s)!
usage: racon [options ...] <sequences> <overlaps> <target sequences>

    #default output is stdout
    <sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences used for correction
    <overlaps>
        input file in MHAP/PAF/SAM format (can be compressed with gzip)
        containing overlaps between sequences and target sequences
    <target sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences which will be corrected

    options:
        -u, --include-unpolished
            output unpolished target sequences
        -f, --fragment-correction
            perform fragment correction instead of contig polishing
            (overlaps file should contain dual/self overlaps!)
        -w, --window-length <int>
            default: 500
            size of window on which POA is performed
        -q, --quality-threshold <float>
    

: 1

## Racon first round

In [9]:

mkdir -p ~/SG-ONT-2021/RESULTS/FLYE_RACON
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON
time minimap2 -t 4 ../FLYE/assembly.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon1.paf
time racon -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon1.paf ../FLYE/assembly.fasta > assembly.racon1.fasta

(racon) (racon) (racon) [M::mm_idx_gen::0.090*1.04] collected minimizers
[M::mm_idx_gen::0.114*1.62] sorted minimizers
[M::main::0.114*1.62] loaded/built the index for 4 target sequence(s)
[M::mm_mapopt_update::0.119*1.59] mid_occ = 5
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 4
[M::mm_idx_stat::0.123*1.57] distinct minimizers: 226983 (98.75% are singletons); average occurrences: 1.016; average spacing: 5.344
[M::worker_pipeline::6.620*2.49] mapped 11235 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 4 ../FLYE/assembly.fasta ../../DATA/Clone10/ONT/Clone10.fastq.gz
[M::main] Real time: 6.630 sec; CPU: 16.522 sec; Peak RSS: 0.179 GB

real	0m6.659s
user	0m16.298s
sys	0m0.239s
(racon) [racon::Polisher::initialize] loaded target sequences 0.013830 s
[racon::Polisher::initialize] loaded sequences 4.280443 s
[racon::Polisher::initialize] loaded overlaps 0.020306 s
[racon::Polisher::initialize] aligning overlaps [====================] 35.727142 s
[racon::Poli

: 1

## Racon second round

In [15]:
# second round
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON
time minimap2 -t 4 assembly.racon1.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon2.paf
time racon -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon2.paf assembly.racon1.fasta > assembly.racon2.fasta
conda deactivate

(racon) (racon) [M::mm_idx_gen::0.094*1.04] collected minimizers
[M::mm_idx_gen::0.122*1.66] sorted minimizers
[M::main::0.122*1.66] loaded/built the index for 4 target sequence(s)
[M::mm_mapopt_update::0.129*1.62] mid_occ = 5
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 4
[M::mm_idx_stat::0.135*1.60] distinct minimizers: 225895 (98.72% are singletons); average occurrences: 1.017; average spacing: 5.337
[M::worker_pipeline::6.919*2.44] mapped 11235 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 4 assembly.racon1.fasta ../../DATA/Clone10/ONT/Clone10.fastq.gz
[M::main] Real time: 6.930 sec; CPU: 16.874 sec; Peak RSS: 0.180 GB

real	0m6.950s
user	0m16.579s
sys	0m0.313s
(racon) [racon::Polisher::initialize] loaded target sequences 0.013035 s
[racon::Polisher::initialize] loaded sequences 3.822300 s
[racon::Polisher::initialize] loaded overlaps 0.020336 s
[racon::Polisher::initialize] aligning overlaps [====================] 36.110539 s
[racon::Polisher::ini

: 1

# 3. Assemblies correction with Medaka

Correction can improve the consensus sequence for a draft genome assembly.

Medaka uses fast5 files to correct contigs using trained models. These models are freely available.

Medaka allows you to train a model by using fast5 from your favorite species. You can use it directly to obtain a consensus from you favorite organism.

## 1.3 Correct assemblies with Medaka

### 1.3.1 Before to correct assemblies, index them

In [16]:
conda activate medaka
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON

(medaka) (medaka) 

: 1

In [17]:
time samtools faidx assembly.racon2.fasta 
time minimap2 -d assembly.racon2.fasta.mmi assembly.racon2.fasta


real	0m0.018s
user	0m0.015s
sys	0m0.001s
(medaka) [M::mm_idx_gen::0.072*1.04] collected minimizers
[M::mm_idx_gen::0.107*1.65] sorted minimizers
[M::main::0.141*1.49] loaded/built the index for 4 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 4
[M::mm_idx_stat::0.149*1.46] distinct minimizers: 225773 (98.72% are singletons); average occurrences: 1.017; average spacing: 5.337; total length: 1225245
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -d assembly.racon2.fasta.mmi assembly.racon2.fasta
[M::main] Real time: 0.159 sec; CPU: 0.226 sec; Peak RSS: 0.017 GB

real	0m0.163s
user	0m0.152s
sys	0m0.077s
(medaka) 

: 1

In [18]:
#create a medaka repertory
mkdir -p ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA

(medaka) (medaka) (medaka) 

: 1

### 1.3.2 Medaka_consensus

Medaka is a tool to create a consensus sequence from nanopore sequencing data. 

This task is performed by using neural networks applied to a pileup of individual sequencing reads against a draft assembly.

It outperforms graph-based methods operating on basecalled data, and can be competitive with state-of-the-art signal-based methods whilst being much faster.

As input medaka accepts reads in either a .fasta or a .fastq file. It requires a draft assembly as a .fasta.

### Check the usage of medaka_consensus

In [19]:
medaka_consensus


medaka 1.2.2
------------

Assembly polishing via neural networks. The input assembly should be
preprocessed with racon.

medaka_consensus [-h] -i <fastx>

    -h  show this help text.
    -i  fastx input basecalls (required).
    -d  fasta input assembly (required).
    -o  output folder (default: medaka).
    -g  don't fill gaps in consensus with draft sequence.
    -m  medaka model, (default: r941_min_high_g360).
        Available: r103_min_high_g345, r103_min_high_g360, r103_prom_high_g360, r103_prom_snp_g3210, r103_prom_variant_g3210, r10_min_high_g303, r10_min_high_g340, r941_min_fast_g303, r941_min_high_g303, r941_min_high_g330, r941_min_high_g340_rle, r941_min_high_g344, r941_min_high_g351, r941_min_high_g360, r941_prom_fast_g303, r941_prom_high_g303, r941_prom_high_g330, r941_prom_high_g344, r941_prom_high_g360, r941_prom_high_g4011, r941_prom_snp_g303, r941_prom_snp_g322, r941_prom_snp_g360, r941_prom_variant_g303, r941_prom_variant_g322, r941_prom_variant_g360.
        Alte

: 1

### Check the medaka model to use

Medaka models are named to indicate i) the pore type, ii) the sequencing device (MinION or PromethION), iii) the basecaller variant, and  iv) the basecaller version

{pore}_{device}_{caller variant}_{caller version}

examples: 

r941_min_fast_g303 : MiniON R9.4.1 flowcells using the fast Guppy basecaller version 3.0.3. 


In [20]:
medaka tools list_models

Available: r103_min_high_g345, r103_min_high_g360, r103_prom_high_g360, r103_prom_snp_g3210, r103_prom_variant_g3210, r10_min_high_g303, r10_min_high_g340, r941_min_fast_g303, r941_min_high_g303, r941_min_high_g330, r941_min_high_g340_rle, r941_min_high_g344, r941_min_high_g351, r941_min_high_g360, r941_prom_fast_g303, r941_prom_high_g303, r941_prom_high_g330, r941_prom_high_g344, r941_prom_high_g360, r941_prom_high_g4011, r941_prom_snp_g303, r941_prom_snp_g322, r941_prom_snp_g360, r941_prom_variant_g303, r941_prom_variant_g322, r941_prom_variant_g360
Default consensus:  r941_min_high_g360
Default snp:  r941_prom_snp_g360
Default variant:  r941_prom_variant_g360
(medaka) 

: 1

### Check the medaka model to use

In [24]:
# go to medaka repertory results
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA
#rm -rf ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS

(medaka) (medaka) (medaka) 

: 1

### Medaka in FLYE + RACON

We run medaka on the FLYE + polished with racon assembly.

In [28]:
# medaka_consensus -i ${BASECALLS} -d ${DRAFT} -o ${OUTDIR} -t ${NPROC} -m r941_min_high_g360
time medaka_consensus -i ~/SG-ONT-2021/DATA/${CLONE}/ONT/${CLONE}.fastq.gz -d ~/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta -o MEDAKA_CONSENSUS -t 4 -m r941_min_high_g360

(medaka) Checking program versions
This is medaka 1.2.2
Program    Version    Required   Pass     
bcftools   1.11       1.9        True     
bgzip      1.11       1.9        True     
minimap2   2.22       2.11       True     
samtools   1.11       1.9        True     
tabix      1.11       1.9        True     
Aligning basecalls to draft
Removing previous index file /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta.mmi
Removing previous index file /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta.fai
Constructing minimap index.
[M::mm_idx_gen::0.083*1.02] collected minimizers
[M::mm_idx_gen::0.120*1.60] sorted minimizers
[M::main::0.151*1.47] loaded/built the index for 4 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 4
[M::mm_idx_stat::0.159*1.44] distinct minimizers: 225773 (98.72% are singletons); average occurrences: 1.017; average spacing: 5.337; total length: 1225245
[M::main] Version: 2.22-r1101
[M::main] CMD: minim

: 1


## Improve parallelism for scaling to large datasets

The medaka_consensus program is good for simple datasets but may not be optimal for running large datasets.

Check complemental documentation [here](https://nanoporetech.github.io/medaka/installation.html#improving-parallelism)

Three steps are needed to scale :

```
# align reads to assembly
mini_align -i basecalls.fasta -r assembly.fasta -P -m \
    -p calls_to_draft.bam -t <threads>

# run lots of jobs like this, change model as appropriate
mkdir results
medaka consensus calls_to_draft.bam results/contigs1-4.hdf \
    --model r941_min_fast_g303 --batch 200 --threads 8 \
    --region contig1 contig2 contig3 contig4
...

# wait for jobs, then collate results
medaka stitch results/*.hdf polished.assembly.fasta
```
**Note : medaka_consensus is not medaka consensus !**

### Conclusion

1. Here, we have obtained a pseudomolecule of a clone by using Flye.

2. We have polished it twice with racon.

3. We have corrected it with medaka models.

You can do similar pipeline using the RAVEN assembler with your favorite Clone.

We will compare the results in next practical "Quality Assemblies". 